In [1]:
# install Requirements.txt

In [3]:
import getpass
import os
from dotenv import load_dotenv

In [6]:
# load env and keys for langsmith and groq
# Prior to this you are expected to visit langsmith and groq websites and get your API keys ready
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"



if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")


In [7]:
from langchain.chat_models import init_chat_model

In [8]:
# make sure you are accessing a valid model, feel free to change the model name
model = init_chat_model("Llama-3.3-70B-Versatile", model_provider="groq")

In [9]:
from langchain_core.messages import HumanMessage

In [11]:
model.invoke([HumanMessage(content="Hi! I'm sagar")])

AIMessage(content="Hi Sagar! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 41, 'total_tokens': 68, 'completion_time': 0.048449265, 'prompt_time': 0.001911595, 'queue_time': 0.049236365, 'total_time': 0.05036086}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9df977d3-c448-48d5-949d-7b8895dc7368-0', usage_metadata={'input_tokens': 41, 'output_tokens': 27, 'total_tokens': 68})

In [12]:
model.invoke([HumanMessage('what is my name?')])

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to know your personal details or identity. Our conversation just started, and I don't retain any information about individual users. If you'd like to share your name with me, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 40, 'total_tokens': 108, 'completion_time': 0.134597567, 'prompt_time': 0.001963172, 'queue_time': 0.048522398, 'total_time': 0.136560739}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_34d416ee39', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--72bc75fc-05da-4d77-b300-67c16d1e8d3a-0', usage_metadata={'input_tokens': 40, 'output_tokens': 68, 'total_tokens': 108})

In [14]:
# models response is not based on the previous conversation, as it is not taking previous conversation as input we will now pass entire conversation as input to model

In [15]:
from langchain_core.messages import AIMessage

In [17]:
model.invoke(
    [
        HumanMessage("Hi, My name is Sagar"),
        AIMessage("Hello Sagar! How can I assist today?"),
        HumanMessage("What is my name?")
    ]
)

AIMessage(content='Your name is Sagar.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 67, 'total_tokens': 74, 'completion_time': 0.019491138, 'prompt_time': 0.003237386, 'queue_time': 0.049232463, 'total_time': 0.022728524}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a686463-6a02-4bda-8789-aa914d0b2e31-0', usage_metadata={'input_tokens': 67, 'output_tokens': 7, 'total_tokens': 74})

Message persistence

In [19]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [20]:
# define new workflow
workflow = StateGraph(state_schema=MessagesState)

In [21]:
# define function to call model
def model_call(state: MessagesState):
    resposne = model.invoke(state['messages'])
    return {'messages': resposne}

In [23]:
# define the nodes in graph
workflow.add_edge(START, "model")
workflow.add_node("model", model_call)

In [24]:
# add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [27]:
# create configurable for passing it to models, It supports multiple conversations threads with single application used by multiple users

config = {"configurable":{"thread_id":"abc123"}}

In [28]:
input_message = [HumanMessage('Hi, My name is Sagar')]
output = app.invoke({"messages": input_message}, config)
print(output)

{'messages': [HumanMessage(content='Hi, My name is Sagar', additional_kwargs={}, response_metadata={}, id='27b3c1bf-a974-47a8-a76f-14ffc8aac93c'), AIMessage(content="Hello Sagar! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 42, 'total_tokens': 69, 'completion_time': 0.040832778, 'prompt_time': 0.001959306, 'queue_time': 0.04871813, 'total_time': 0.042792084}, 'model_name': 'Llama-3.3-70B-Versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--857bd558-af55-4e1d-a087-5044a1efd2e4-0', usage_metadata={'input_tokens': 42, 'output_tokens': 27, 'total_tokens': 69})]}


In [30]:
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hello Sagar! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [32]:
input_message = [HumanMessage('what is my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

Your name is still Sagar. You told me that when we started chatting.
None


In [33]:
# out models now started remembering things, now if we change config it will start a fresh chat

In [35]:
config = {"configurable": {"thread_id":"abc12345"}}
input_message = [HumanMessage('what is my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm here to help answer your questions and provide assistance, but I don't have any personal knowledge about you. Would you like to tell me your name?
None


In [38]:
# let me check if model remembers out conversation with previous thread id
config = {"configurable": {"thread_id":"abc123"}}
input_message = [HumanMessage('Do you remember my name?')]
output = app.invoke({"messages": input_message}, config)
print(output['messages'][-1].pretty_print())

================================== Ai Message ==================================

Yes, I remember your name. You're Sagar.
None
